In [14]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os import path

import src.lib.utility_classfier_tuning as uclf
import src.lib.optimal_threhold_related as thres
import src.lib.fairness_tests as fair

from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import sklearn.preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer

In [15]:
result_path='/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/deep learning/results/'
filename = "LSTM gender prediction_0.csv"
prediction = pd.read_csv(path.join(result_path, filename))

In [16]:
y_test_score = prediction['test_score'][prediction['test_score'].notna()]   
y_test_score_male = prediction['test_male_score'][prediction['test_male_score'].notna()]   
y_test_score_female = prediction['test_female_score'][prediction['test_female_score'].notna()]

In [17]:
y_test_score_male

0       0.190115
1       0.140922
2       0.172150
3       0.182603
4       0.015570
          ...   
7702    0.163235
7703    0.077149
7704    0.032254
7705    0.020503
7706    0.039099
Name: test_male_score, Length: 7707, dtype: float64

In [24]:
def get_result (records, auc_vec, ba_vec, eod_vec, di_vec, y_test, y_score, y_test_1, y_score_1, y_test_2, y_score_2):    
    threshold = 0.1
    ba = thres.calculate_balanced_accuracy(y_test, y_score, threshold)
    auroc = roc_auc_score(y_test, y_score)
    precision, recall, tpr, tnr, pd_overall = thres.calculate_precision_metrics(y_test, y_score,threshold)
    
    ba_male = thres.calculate_balanced_accuracy(y_test_1, y_score_1, threshold)
    precision_male, recall_male, tpr_male, tnr_male, pd_male = thres.calculate_precision_metrics(y_test_1, y_score_1,threshold)
    
    ba_female = thres.calculate_balanced_accuracy (y_test_2, y_score_2, threshold)
    precision_female, recall_female, tpr_female, tnr_female, pd_female = thres.calculate_precision_metrics(y_test_2, y_score_2,threshold)

    eod = fair.get_EOD(y_test_1, y_score_1,threshold, y_test_2, y_score_2, threshold)
    di = fair.get_SP(y_test_1, y_score_1,threshold, y_test_2, y_score_2, threshold)
    
    auc_vec.append(auroc)
    ba_vec.append(ba)
    eod_vec.append(eod)
    di_vec.append(di)

    records.append({
        'auroc': auroc,
        'overall ba test': ba,
        'male ba test': ba_male,
        'female ba test': ba_female,
        'overall precision':precision,
        'overall recall':recall,
        'overall tpr':tpr,
        'overall tnr':tnr,
        'overall pd':pd_overall,
        'male precision':precision_male,
        'male recall':recall_male,
        'male tpr':tpr_male,
        'male tnr':tnr_male,
        'male pd':pd_male,
        'female precision':precision_female,
        'female recall':recall_female,
        'female tpr':tpr_female,
        'female tnr':tnr_female,
        'female pd':pd_female,
        'eod': eod,
        'di': di,
        })

In [19]:
records_lstm_race = []
auc_race = []
ba_race = []
eod_race = []
di_race = []
for i in range(5):
    filename = "LSTM race prediction_" + str(i) + ".csv"
    prediction = pd.read_csv(path.join(result_path, filename))
    
    y_test_score = prediction['test_score'][prediction['test_score'].notna()]   
    y_test_score_white = prediction['test_white_score'][prediction['test_white_score'].notna()]   
    y_test_score_black = prediction['test_black_score'][prediction['test_black_score'].notna()]
    
    filename_2 = "LSTM result_" + str(i) + ".csv"
    result = pd.read_csv(path.join(result_path, filename_2))
    y_test = result['y_test'][result['y_test'].notna()] 
    y_test_white = result['y_test_white'][result['y_test_white'].notna()] 
    y_test_black = result['y_test_black'][result['y_test_black'].notna()] 
    
    get_result(records_lstm_race, auc_race, ba_race, eod_race, di_race, y_test, y_test_score, y_test_white, y_test_score_white, y_test_black, y_test_score_black)
    

True positive rate of class 1 is  0.711
True positive rate of class 2 is  0.57
Positive prediction rate of class 1 is  0.388
Positive prediction rate of class 2 is  0.292
True positive rate of class 1 is  0.732
True positive rate of class 2 is  0.556
Positive prediction rate of class 1 is  0.383
Positive prediction rate of class 2 is  0.27
True positive rate of class 1 is  0.71
True positive rate of class 2 is  0.581
Positive prediction rate of class 1 is  0.372
Positive prediction rate of class 2 is  0.29
True positive rate of class 1 is  0.769
True positive rate of class 2 is  0.736
Positive prediction rate of class 1 is  0.421
Positive prediction rate of class 2 is  0.344
True positive rate of class 1 is  0.744
True positive rate of class 2 is  0.67
Positive prediction rate of class 1 is  0.409
Positive prediction rate of class 2 is  0.322


In [20]:
print(auc_race, np.mean(auc_race), np.std(auc_race))
print(ba_race, np.mean(ba_race), np.std(ba_race))
print(eod_race, np.mean(eod_race), np.std(eod_race))
print(di_race, np.mean(di_race), np.std(di_race))

[0.7317515545699996, 0.745816040171713, 0.7461133861376896, 0.759811045995653, 0.7519486982232731] 0.7470881450196657 0.009200832559424619
[0.6741392726343165, 0.6878810240879287, 0.6831939051890283, 0.6950820333062526, 0.6851265459879972] 0.6850845562411048 0.0067991960974672205
[0.14100000000000001, 0.17599999999999993, 0.129, 0.03300000000000003, 0.07399999999999995] 0.11059999999999999 0.05079606283955478
[1.3287671232876714, 1.4185185185185185, 1.2827586206896553, 1.2238372093023255, 1.2701863354037266] 1.3048135614403795 0.06594728984486171


In [25]:
records_lstm_gender = []
auc_gender = []
ba_gender = []
eod_gender = []
di_gender = []
for i in range(5):
    filename = "LSTM gender prediction_" + str(i) + ".csv"
    prediction = pd.read_csv(path.join(result_path, filename))
    
    y_test_score = prediction['test_score'][prediction['test_score'].notna()]   
    y_test_score_male = prediction['test_male_score'][prediction['test_male_score'].notna()]   
    y_test_score_female = prediction['test_female_score'][prediction['test_female_score'].notna()]
    
    filename_2 = "LSTM result_" + str(i) + ".csv"
    result = pd.read_csv(path.join(result_path, filename_2))
    y_test = result['y_test'][result['y_test'].notna()] 
    y_test_male = result['y_test_male'][result['y_test_male'].notna()] 
    y_test_female = result['y_test_female'][result['y_test_female'].notna()] 
    
    get_result(records_lstm_gender, auc_gender, ba_gender, eod_gender, di_gender, y_test, y_test_score, y_test_male, y_test_score_male, y_test_female, y_test_score_female)
    

True positive rate of class 1 is  0.798
True positive rate of class 2 is  0.612
Positive prediction rate of class 1 is  0.523
Positive prediction rate of class 2 is  0.295
True positive rate of class 1 is  0.75
True positive rate of class 2 is  0.68
Positive prediction rate of class 1 is  0.445
Positive prediction rate of class 2 is  0.325
True positive rate of class 1 is  0.775
True positive rate of class 2 is  0.63
Positive prediction rate of class 1 is  0.474
Positive prediction rate of class 2 is  0.299
True positive rate of class 1 is  0.835
True positive rate of class 2 is  0.711
Positive prediction rate of class 1 is  0.513
Positive prediction rate of class 2 is  0.354
True positive rate of class 1 is  0.786
True positive rate of class 2 is  0.698
Positive prediction rate of class 1 is  0.474
Positive prediction rate of class 2 is  0.355


In [26]:
print(auc_gender, np.mean(auc_gender), np.std(auc_gender))
print(ba_gender, np.mean(ba_gender), np.std(ba_gender))
print(eod_gender, np.mean(eod_gender), np.std(eod_gender))
print(di_gender, np.mean(di_gender), np.std(di_gender))

[0.7317515545699996, 0.745816040171713, 0.7461133861376896, 0.759811045995653, 0.7519486982232731] 0.7470881450196657 0.009200832559424619
[0.6741392726343165, 0.6878810240879287, 0.6831939051890283, 0.6950820333062526, 0.6851265459879972] 0.6850845562411048 0.0067991960974672205
[0.18600000000000005, 0.06999999999999995, 0.14500000000000002, 0.124, 0.08800000000000008] 0.12260000000000001 0.041200000000000014
[1.7728813559322036, 1.3692307692307693, 1.585284280936455, 1.4491525423728815, 1.3352112676056338] 1.5023520432155886 0.16034423247567478
